# Примеры задач анализа текстов
Классификация текстов: спам-фильтр для SMS

В этом задании вам предстоит взять открытый датасет с SMS-сообщениями, размеченными на спам ("spam") и не спам ("ham"), построить на нем классификатор текстов на эти два класса, оценить его качество с помощью кросс-валидации, протестировать его работу на отдельных примерах, и посмотреть, что будет происходить с качеством, если менять параметры вашей модели.

Задание

1. Загрузите датасет. Описание датасета можно посмотреть здесь.
smsspamcollection.zip
2. Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

3. Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

4. Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.

5. Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

6. А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

"FreeMsg: Txt: claim your reward of 3 hours talk time"

"Have you visited the last lecture on physics?"

"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

"Only 99$"

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

7. Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

8. Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

По какой-то причине обучение наивного байесовского классификатора через Pipeline происходит с ошибкой. Чтобы получить правильный ответ, отдельно посчитайте частоты слов и обучите классификатор.

9. Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

1. Загрузите датасет. Описание датасета можно посмотреть здесь.  
smsspamcollection.zip

2. Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
with open ('data/SMSSpamCollection.txt', encoding='utf8') as x:
    data = x.read()
data = np.array(data.split(sep='\n'))
data

array(['ham\tGo until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
       'ham\tOk lar... Joking wif u oni...',
       "spam\tFree entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
       ...,
       "ham\tThe guy did some bitching but I acted like i'd be interested in buying something else next week and he gave it to us for free",
       'ham\tRofl. Its true to its name', ''], dtype='<U914')

3. Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [3]:
metki = []
texts = []
for i in data:
    i = i.split(sep='\t')
    if len(i) == 2:
        metki += [i[0]]
        texts += [i[1]]

In [4]:
print(len(metki), len(texts))

5574 5574


In [5]:
df = pd.DataFrame()
df['metki'] = metki
df['texts'] = texts
df

,metki,texts
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [6]:
df['metki_fct'] = df['metki'].factorize()[0]
df

,metki,texts,metki_fct
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0
5,spam,FreeMsg Hey there darling it's been 3 week's n...,1
6,ham,Even my brother is not like to speak with me. ...,0
7,ham,As per your request 'Melle Melle (Oru Minnamin...,0
8,spam,WINNER!! As a valued network customer you have...,1
9,spam,Had your mobile 11 months or more? U R entitle...,1


4. Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
txts = CountVectorizer()
X = txts.fit_transform(df['texts'])
print(txts.get_feature_names()[1000:1100])
print(X.shape, X.toarray())

['ahhhh', 'ahmad', 'ahold', 'aid', 'aids', 'aig', 'aight', 'ain', 'aint', 'air', 'air1', 'airport', 'airtel', 'aiya', 'aiyah', 'aiyar', 'aiyo', 'ajith', 'ak', 'aka', 'akon', 'al', 'alaikkum', 'alaipayuthe', 'albi', 'album', 'alcohol', 'aldrine', 'alert', 'alertfrom', 'alerts', 'aletter', 'alex', 'alfie', 'algarve', 'algebra', 'algorithms', 'ali', 'alian', 'alibi', 'alive', 'all', 'allah', 'allalo', 'allday', 'alle', 'allo', 'allow', 'allowed', 'allows', 'almost', 'alone', 'along', 'alot', 'already', 'alright', 'alrite', 'also', 'alter', 'alternative', 'although', 'alto18', 'aluable', 'alwa', 'always', 'alwys', 'am', 'amanda', 'amazing', 'ambitious', 'ambrith', 'american', 'ami', 'amigos', 'amk', 'amla', 'amma', 'ammae', 'ammo', 'amnow', 'among', 'amongst', 'amore', 'amount', 'amp', 'amplikater', 'amrca', 'amrita', 'ams', 'amt', 'amused', 'amy', 'an', 'ana', 'anal', 'analysis', 'anand', 'and', 'anderson', 'andre']
(5574, 8713) [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ..

5. Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

LR = LogisticRegression(random_state=2)
scores = cross_val_score(LR, X, df['metki_fct'], cv=10,scoring='f1')
answer1 = round(scores.mean(),1)
answer1

0.9

6. А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

"FreeMsg: Txt: claim your reward of 3 hours talk time"

"Have you visited the last lecture on physics?"

"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

"Only 99$"

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [9]:
new_msg = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
           "FreeMsg: Txt: claim your reward of 3 hours talk time",
           "Have you visited the last lecture on physics?",
           "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
           "Only 99$"]

In [10]:
LR.fit(X,df['metki_fct'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=2, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [11]:
answer2 = []
for i in new_msg:
    answer2 += [int(LR.predict(txts.transform([i])))]
answer2

[1, 1, 0, 0, 0]

7. Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [12]:
answer3 = []

txts_22 = CountVectorizer(ngram_range=(2, 2))
X_22 = txts_22.fit_transform(df['texts'])
score_22 = cross_val_score(LR,X_22, df['metki_fct'], cv=10, scoring='f1')
answer3 += [round(score_22.mean(),2)]

txts_33 = CountVectorizer(ngram_range=(3, 3))
X_33 = txts_33.fit_transform(df['texts'])
score_33 = cross_val_score(LR,X_33, df['metki_fct'], cv=10, scoring='f1')
answer3 += [round(score_33.mean(),2)]

txts_13 = CountVectorizer(ngram_range=(1, 3))
X_13 = txts_13.fit_transform(df['texts'])
score_13 = cross_val_score(LR,X_13, df['metki_fct'], cv=10, scoring='f1')
answer3 += [round(score_13.mean(),2)]

answer3 # возможно так 0.82 0.72 0.92

[0.82, 0.73, 0.93]

8. Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.  

По какой-то причине обучение наивного байесовского классификатора через Pipeline происходит с ошибкой. Чтобы получить правильный ответ, отдельно посчитайте частоты слов и обучите классификатор.



In [13]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()

mnb_score_22 = cross_val_score(mnb,X_22, df['metki_fct'], cv=10, scoring='f1')
print(round(mnb_score_22.mean(),2))

mnb_score_33 = cross_val_score(mnb,X_33, df['metki_fct'], cv=10, scoring='f1')
print(round(mnb_score_33.mean(),2))

mnb_score_13 = cross_val_score(mnb,X_13, df['metki_fct'], cv=10, scoring='f1')
print(round(mnb_score_13.mean(),2))
answer4 = [round(mnb_score_22.mean(),2),round(mnb_score_33.mean(),2),round(mnb_score_13.mean(),2)]

0.65
0.38
0.89


9. Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [15]:
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(df['texts'])

In [16]:
X_tfidf.shape

(5574, 8713)

In [17]:
LR = LogisticRegression(random_state=2)
tfidf_scores = cross_val_score(LR, X_tfidf, df['metki_fct'], cv=10,scoring='f1')
print(scores.mean(), tfidf_scores.mean())
answer5 = '-1'

0.9333485268579415 0.8528599554172456


In [18]:
def write_answer1(answer):
    with open('answers/5.3. Text analysis_answer1.txt',"w") as fout:
        fout.write(str(answer))
write_answer1(answer1)
        
def write_answer2(answer):
    with open('answers/5.3. Text analysis_answer2.txt',"w") as fout:
        fout.write(' '.join(str(i) for i in answer))
write_answer2(answer2)
        
def write_answer3(answer):
    with open('answers/5.3. Text analysis_answer3.txt',"w") as fout:
        fout.write(' '.join(str(i) for i in answer))
write_answer3(answer3)
        
def write_answer4(answer):
    with open('answers/5.3. Text analysis_answer4.txt',"w") as fout:
        fout.write(' '.join(str(i) for i in answer))
write_answer4(answer4)

def write_answer5(answer):
    with open('answers/5.3. Text analysis_answer5.txt',"w") as fout:
        fout.write(str(answer))      
write_answer5(answer5)

print(answer1)
print(answer2)
print(answer3)
print(answer4)
print(answer5)

0.9
[1, 1, 0, 0, 0]
[0.82, 0.73, 0.93]
[0.65, 0.38, 0.89]
-1
